TODO:
1. tokenize (from old project)
2. lemmatize? (see if can use eliana's code)
3. remove stopwords
4. feature-generator - for cell ("speaker interval"), for full protocol, and for list of protocols
5. look for word embedding
6. understand deep LDA from reference (possibly different embedding)

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import re
import pickle
from pathlib import Path
from time import time
from datetime import datetime
from tqdm import tqdm, tnrange, tqdm_notebook
from pprint import pprint
from warnings import warn
from datetime import datetime
import itertools
from collections import Counter, OrderedDict
from bidi import algorithm as bidi
from hebrew_stopwords import hebrew_stopwords

In [ ]:
mpl.rcParams.update({'font.size': 12})

# Load data

In [ ]:
DATA = Path('Data')
meta = pd.read_csv(DATA/'meta.csv')
df = pd.read_csv(DATA/'data.csv')

# Parser

In [ ]:
# class Parser:
#     def __init__(self):
#         self.
        

# EDA

In [ ]:
def show_freqs(tokens, vocabulary=None, n=15, logscale=True, token_lab='Token', compact=False):
    if compact:
        _, axs = plt.subplots(1,3, figsize=(18,5))
    else:
        _, axs = plt.subplots(3,1, figsize=(12,12))
    
    if vocabulary:
        vocabulary = set(vocabulary)
        tokens = [token for token in  tokens if token in vocabulary]
    count = Counter(tokens)
    count = OrderedDict(sorted(count.items(), key=lambda kv: kv[1]))
    vals = list(count.values())
    keys = list(count.keys())
    bidi_keys = [bidi.get_display(token) for token in keys]
    
    # quantile plot
    ax = axs[0]
    ax.axhline(np.sum(vals)/len(vals), linestyle=':', color='blue', label='Average')
    ax.plot(list(range(101)), [vals[int(q/100*(len(vals)-1))] for q in range(101)], 'k.-')
    ax.set_xlabel('Quantile [%]')
    ax.set_ylabel('Number of occurences')
    ax.set_xlim((0,100))
    ax.set_title(f'Total: {len(tokens):d}, Unique: {len(keys):d}')
    if logscale:
        ax.set_yscale('log')
    ax.grid()
    ax.legend()
    
    # top tokens
    ax = axs[1]
    ax.bar(bidi_keys[-1:-n-1:-1], vals[-1:-n-1:-1])
    ax.tick_params(axis='x', rotation=45)
    ax.set_xlabel(token_lab)
    ax.set_ylabel('Occurences')
    ax.set_title('Top')
    if logscale:
        ax.set_yscale('log')
    ax.grid()
    for label in ax.xaxis.get_ticklabels():
        label.set_ha('right')
    
    # tail tokens
    ax = axs[2]
    ax.bar(list(reversed(bidi_keys[:n])), list(reversed(vals[:n])))
    ax.tick_params(axis='x', rotation=45)
    for label in ax.xaxis.get_ticklabels():
        label.set_ha('right')
    ax.set_xlabel(token_lab)
    ax.set_ylabel('Occurences')
    ax.set_title('Tail')
    if logscale:
        ax.set_yscale('log')
    ax.grid()
    
    plt.tight_layout()
    
    return (list(reversed(keys)), list(reversed(vals)))